In [6]:
#import libraries

import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
#library to create a dictionaty
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
import re

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
from collections import defaultdict
import itertools

In [9]:
#import training data

df = pd.read_csv("C:/Users/270168/Downloads/personal/nlp-starter-test/social_media_clean_text.csv")

In [10]:
#check the data set

df.head()

,text,choose_one,class_label
0,just happened a terrible car crash,Relevant,1
1,our deeds are the reason of this earthquake m...,Relevant,1
2,"heard about earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,forest fire near la ronge sask canada,Relevant,1


In [11]:
#separate text column for preprocessing

df["processed_text"] = df.iloc[:,0]

In [12]:
def process (G):
    tok = G.split(" ")
    stop_words = stopwords.words('english')
    no_tok = [t for t in tok if t not in stop_words]
    filtered_tok = [tok1 for tok1 in no_tok if re.search('[a-zA-Z]', tok1)]
    word_lemmat = WordNetLemmatizer()
    lemmat_words = [word_lemmat.lemmatize(t) for t in filtered_tok]
    return lemmat_words

In [13]:
df["processed_text"] = df["processed_text"].apply(process)

In [14]:
data_text = df["processed_text"].values.tolist()

In [15]:
dictionary = Dictionary(df["processed_text"])

In [16]:
dictionary.token2id.get("flood")

72

In [17]:
corpus = [dictionary.doc2bow(t) for t in data_text]

In [18]:
gross_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    gross_count[word_id] += word_count

In [19]:
sorted_gross_count = sorted(gross_count.items(), key=lambda w: w[1], reverse=True)

In [20]:
#print top 10 words

for word_id, word_count in sorted_gross_count[:10]:
    print(dictionary.get(word_id), word_count)

fire 443
amp 442
like 436
i'm 309
u 307
get 303
new 268
one 253
people 245
via 235


In [21]:
#create tfidf model

tfidf = TfidfModel(corpus)


In [22]:
dictionary.values()

ValuesView(<gensim.corpora.dictionary.Dictionary object at 0x000002B68B9FF048>)

In [23]:
from sklearn import feature_extraction

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.01, use_idf=True, tokenizer=process, ngram_range=(1,3))

In [26]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text"])

In [27]:
print(tfidf_matrix.shape)

(9282, 78)


In [28]:
terms = tfidf_vectorizer.get_feature_names()

In [29]:
print(terms)

['accident', 'amp', 'attack', 'back', 'body', 'bomb', 'building', 'burning', 'california', "can't", 'car', 'collapse', 'could', 'crash', 'day', 'dead', 'death', 'disaster', 'emergency', 'even', 'fire', 'first', 'flood', 'full', 'get', 'go', 'going', 'good', 'got', 'hiroshima', 'home', 'house', "i'm", 'injury', 'killed', 'know', 'last', 'life', 'like', 'look', 'love', 'make', 'man', 'mass', 'need', 'never', 'new', 'news', 'nuclear', 'one', 'people', 'police', 'right', 'rt', 'say', 'see', 'service', 'still', 'storm', 'suicide', 'take', 'think', 'time', 'today', 'train', 'two', 'u', 'via', 'video', 'w', 'want', 'war', 'way', 'weapon', 'woman', 'world', 'would', 'year']


In [30]:
#K Means Cluster

from sklearn.cluster import KMeans
n_cluster = 5
km = KMeans(n_clusters=n_cluster)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [31]:
df['cluster'] = clusters

In [32]:
df['cluster'].value_counts()

0    8192
1     376
4     274
3     220
2     220
Name: cluster, dtype: int64

In [33]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]



In [34]:
from sklearn.decomposition import LatentDirichletAllocation

In [35]:
lda_model = LatentDirichletAllocation(n_components=2, max_iter=10, learning_method='online',learning_offset=50).fit(tfidf_matrix)

In [36]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [37]:
pyLDAvis.sklearn.prepare(lda_model,tfidf_matrix, tfidf_vectorizer)

C:\ProgramData\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:109: DeprecationWarning: invalid escape sequence \d
  nbits = re.compile('(\d+)bit').search(abits).group(1)
C:\ProgramData\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:662: DeprecationWarning: invalid escape sequence \s
  "\s+stepping\s+(?P<STP>\d+)", re.IGNORECASE)


PreparedData(topic_coordinates=           Freq  cluster  topics         x    y
topic                                          
1      51.40349        1       1  0.329088  0.0
0      48.59651        1       2 -0.329088  0.0, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
20    Default  283.000000        fire  283.000000  30.0000  30.0000
38    Default  273.000000        like  273.000000  29.0000  29.0000
1     Default  245.000000         amp  245.000000  28.0000  28.0000
24    Default  205.000000         get  205.000000  27.0000  27.0000
32    Default  191.000000         i'm  191.000000  26.0000  26.0000
47    Default  172.000000        news  172.000000  25.0000  25.0000
49    Default  165.000000         one  165.000000  24.0000  24.0000
46    Default  175.000000         new  175.000000  23.0000  23.0000
66    Default  175.000000           u  175.000000  22.0000  22.0000
67    Default  172.000000         via  172.000000  21.0000  21.0000
58    Default  146.000000       storm  146.000000  20.0000  20.0000
68    Default  145.000000       video  145.000000  19.0000  19.0000
50    Default  152.000000      people  152.000000  18.0000  18.0000
76    Default  122.000000       would  122.000000  17.0000  17.0000
77    Default  119.000000        year  119.000000  16.0000  16.0000
30    Default  116.000000        home  116.000000  15.0000  15.0000
62    Default  126.000000        time  126.000000  14.0000  14.0000
18    Default  126.000000   emergency  126.000000  13.0000  13.0000
25    Default  113.000000          go  113.000000  12.0000  12.0000
53    Default  111.000000          rt  111.000000  11.0000  11.0000
17    Default  109.000000    disaster  109.000000  10.0000  10.0000
59    Default  118.000000     suicide  118.000000   9.0000   9.0000
51    Default  110.000000      police  110.000000   8.0000   8.0000
54    Default  110.000000         say  110.000000   7.0000   7.0000
28    Default   99.000000         got   99.000000   6.0000   6.0000
40    Default   98.000000        love   98.000000   5.0000   5.0000
48    Default   98.000000     nuclear   98.000000   4.0000   4.0000
14    Default  107.000000         day  107.000000   3.0000   3.0000
21    Default   97.000000       first   97.000000   2.0000   2.0000
8     Default   95.000000  california   95.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
38     Topic2  272.887655        like  273.465090   0.7195  -2.6950
24     Topic2  204.837668         get  205.413083   0.7188  -2.9819
47     Topic2  172.110746        news  172.692851   0.7182  -3.1560
49     Topic2  164.675983         one  165.269029   0.7180  -3.2001
68     Topic2  145.096986       video  145.649723   0.7178  -3.3267
58     Topic2  145.828532       storm  146.398935   0.7177  -3.3217
76     Topic2  121.572046       would  122.158813   0.7168  -3.5036
30     Topic2  115.655625        home  116.217094   0.7168  -3.5535
77     Topic2  119.040421        year  119.647279   0.7165  -3.5246
25     Topic2  112.482222          go  113.055987   0.7165  -3.5813
53     Topic2  110.545050          rt  111.109673   0.7165  -3.5987
17     Topic2  108.747378    disaster  109.309927   0.7165  -3.6151
8      Topic2   95.082308  california   95.612225   0.7161  -3.7493
40     Topic2   97.779037        love   98.346398   0.7158  -3.7214
28     Topic2   99.381372         got   99.966769   0.7157  -3.7051
3      Topic2   92.400736        back   92.962708   0.7156  -3.7780
39     Topic2   93.708789        look   94.299195   0.7153  -3.7639
48     Topic2   97.463924     nuclear   98.078198   0.7153  -3.7246
21     Topic2   96.766341       first   97.377337   0.7153  -3.7318
6      Topic2   84.119316    building   84.658646   0.7152  -3.8719
33     Topic2   86.775283      injury   87.363595   0.7149  -3.8408
22     Topic2   86.682100       flood   87.273807   0.7148  -3.8418
42     Topic2   83.587209